Estudando análise de dados no Spotify a partir do tutorial de Análise de Dados e Forró de Dandara Sousa: https://dandaramcsousa.github.io/2020/05/26/dados-e-forro-vol-1.html

In [ ]:
!pip install spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
import sys
import csv
 
sp = spotipy.Spotify(client_credentials_manager = SpotifyClientCredentials(client_id="SUACREDENCIAL", client_secret="SUACREDENCIAL"))

get_artist(name): dado um nome, retorna um artista

In [ ]:
def get_artist(name):
    results = sp.search(name)
    items = results['tracks']['items']
    if len(items) > 0:
        return items[0]['artists'][0]
    else:
        return None

get_artist_id(artists): dado um artista, retorna o id dele

In [ ]:
def get_artist_id(artist):
    return artist['id']

get_artist_albums_id_names(id): dado o id do artista, retorna os ids e nomes de todos os álbuns daquele artista disponíveis no Brasil

In [ ]:
def get_artist_albums_id_names(id):
  albums = sp.artist_albums(id, country = 'BR', limit=50)
  albums_id_name = {}
  for i in range(len(albums['items'])):
    id = albums['items'][i]['id']
    name = albums['items'][i]['name']
    albums_id_name[id] = name
 
  return albums_id_name

get_album_songs(album_id, album_name): dado o id e nome do álbum, retorna um dicionário com as informações das músicas do álbum (álbum, tipo do álbum, número da música no álbum, id, nome, popularidade, se a faixa é explícita, duração em milissegundos, data de lançamento e artistas envolvidos)

In [ ]:
def get_album_songs(album_id, album_name):
  spotify_album = {}
 
  tracks = sp.album_tracks(album_id)
  
  for n in range(len(tracks['items'])):
    id_track = tracks['items'][n]['id']
    track = sp.track(id_track)
    spotify_album[id_track] = {}
    
    spotify_album[id_track]['album'] = album_name
    spotify_album[id_track]['album_type'] = track['album']['album_type']
    spotify_album[id_track]['track_number'] = track['track_number']
    spotify_album[id_track]['id_track'] = track['id']
    spotify_album[id_track]['name'] = track['name']
    spotify_album[id_track]['popularity'] = track['popularity']
    spotify_album[id_track]['explicit'] = track['explicit']
    spotify_album[id_track]['duration_ms'] = track['duration_ms']
    spotify_album[id_track]['release_date'] = track['album']['release_date']
 
    artists_track = track['artists']
    spotify_album[id_track]['artists'] = []
    for artist in artists_track:
      spotify_album[id_track]['artists'].append(artist['name'])
  return spotify_album

get_all_albums_songs(albums_ids_names): dado um dicionário com ids e nomes de álbuns, retorna todas as músicas de todos os albuns de um artista provenientes da função get_album_songs(album_id, album_name)

In [ ]:
def get_all_albums_songs(albums_ids_names):
  spotify_albums = []
  albums_names = []
  for id, name in albums_ids_names.items():
    if name not in albums_names:
      albums_names.append(name)
      album_songs = get_album_songs(id,name) 
    for item in album_songs.items():
      spotify_albums.append(item[1]) 
  return spotify_albums

convert_to_csv(filepath, name): converte para um csv os dados

In [ ]:
def convert_to_csv(filepath, name):
  keys = filepath[0].keys()
  print(keys)
  csv_name = ''+ name + '.csv'
  with open(csv_name, 'w') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(filepath)
  return

script

In [ ]:
name = "Sorriso Maroto"
artist = get_artist(name)    
if artist:
  artist_id = get_artist_id(artist)
  albums_id_names = get_artist_albums_id_names(artist_id)
  all_albums = get_all_albums_songs(albums_id_names)
  convert_to_csv(all_albums, 'sorrisomaroto')   
else:
  logger.error("Can't find artist: %s", artist)

dict_keys(['album', 'album_type', 'track_number', 'id_track', 'name', 'popularity', 'explicit', 'duration_ms', 'release_date', 'artists'])


Começamos carregando nosso conjunto de dados e importando o Altair, uma biblioteca Python para visualização de dados.

In [ ]:
import altair as alt
import pandas as pd
 
data = pd.read_csv('sorrisomaroto.csv')

Top 20 das músicas. Exploramos se o nossas músicas favoritas estão entre as mais populares do Spotify.

In [ ]:
data_top20 = data.sort_values('popularity', ascending = False).head(20)
alt.Chart(data_top20).mark_circle(color = '#222437', opacity = 1).encode(
    y = alt.Y('name', axis = alt.Axis(title='Nome'), sort = None),
    x = alt.X('popularity', axis = alt.Axis(title = "Popularidade")),
    tooltip = ['name', 'popularity', 'album', 'album_type']
)

alt.Chart(...)

Álbuns mais populares. O ‘album_type’ nos informa a categoria daquele álbum, podendo ser um álbum, um single, um EP. Como queremos apenas os álbuns mais populares, vamos restringir ao tipo ‘album’. E dessa vez vamos colocar mais um conceito do Altair: a sobreposição de gráficos. Para isso, basta atribuir cada gráfico a uma variável e depois somá-las.

In [ ]:
data_groupby = pd.DataFrame(data, columns = ['album','album_type','popularity','duration_ms'])
 
data_groupby_album_chart =  data_groupby[data_groupby['album_type'] ==  'album']
 
bars = alt.Chart(data_groupby_album_chart).mark_bar(color = '#222437').encode(
    x=alt.X('median(popularity):Q', axis = alt.Axis(title='Mediana da Popularidade')),
    y=alt.Y('album:O', sort='-x', axis = alt.Axis(title='Álbum'))
)
text = alt.Chart(data_groupby_album_chart).mark_text(dx=-15, dy=3, color='white').encode(
    x=alt.X('median(popularity):Q', axis = alt.Axis(title='Mediana da Popularidade')),
    y=alt.Y('album:O', sort='-x', axis = alt.Axis(title='Álbum')),
    text=alt.Text('median(popularity):Q')
)
bars + text

alt.LayerChart(...)

In [ ]:
data_groupby = pd.DataFrame(data, columns = ['album','album_type','popularity','duration_ms'])
 
data_groupby_album_chart =  data_groupby[data_groupby['album_type'] ==  'single']
 
bars = alt.Chart(data_groupby_album_chart).mark_bar(color = '#222437').encode(
    x=alt.X('median(popularity):Q', axis = alt.Axis(title='Mediana da Popularidade')),
    y=alt.Y('album:O', sort='-x', axis = alt.Axis(title='Single'))
)
text = alt.Chart(data_groupby_album_chart).mark_text(dx=-15, dy=3, color='white').encode(
    x=alt.X('median(popularity):Q', axis = alt.Axis(title='Mediana da Popularidade')),
    y=alt.Y('album:O', sort='-x', axis = alt.Axis(title='Single')),
    text=alt.Text('median(popularity):Q')
)
bars + text

alt.LayerChart(...)

In [ ]:
# linha do tempo de lançamentos
data_time= pd.DataFrame(data, columns = ['name', 'album','album_type','release_date'])
range_ = ['#222437', '#B183AE']
alt.Chart(data_time).mark_circle(opacity = 1).encode(
    y = alt.Y('release_date:T', axis = alt.Axis(title='Lançamento')),
    x = alt.X('album:N', sort='y', axis = alt.Axis(title='Álbum')),
    color = alt.Color('album_type', legend=alt.Legend(title="Tipo do álbum"), scale=alt.Scale(range = range_)),
    tooltip = ['album','album_type','release_date:T']
)

alt.Chart(...)

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id="SUACREDENCIAL",
                                               client_secret="SUACREDENCIAL",
                                               redirect_uri="http://SUACREDENCIAL.com",
                                               scope="user-top-read"))

ranges = ['short_term', 'medium_term', 'long_term']

for sp_range in ranges:
    print("range:", sp_range)
    results = sp.current_user_top_tracks(time_range=sp_range, limit=20)
    for i, item in enumerate(results['items']):
        print(i, item['name'], '//', item['artists'][0]['name'])
    print()

Couldn't read cache at: .cache


range: short_term
Enter the URL you were redirected to: 


SpotifyOauthError: ignored